## NREL National Solar Radiation Database (NSRDB) - HSDS Demo

This notebook demonstrates basic usage of the National Renewable Energy Laboratory (NREL) National Solar Radiation Database (NSRDB) data. The data is provided from Amazon Web Services using the HDF Group's Highly Scalable Data Service (HSDS).

For this to work you must first install h5pyd:

```
pip install --user h5pyd
```

Next you'll need to configure HSDS:

```
hsconfigure
```

and enter at the prompt:

```
hs_endpoint = https://developer.nrel.gov/api/hsds
hs_username = None
hs_password = None
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf
```

*The example API key here is for demonstation and is rate-limited per IP. To get your own API key, visit https://developer.nrel.gov/signup/*

You can also add the above contents to a configuration file at ~/.hscfg

In [ ]:
%matplotlib inline
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree

## Basic Usage

The NSRDB is provided in annual .h5 files and currently spans 1998-2018.  
Each year can be accessed from /nrel/nsrdb/nsrdb_${year}.h5

In [ ]:
# Open the desired year of nsrdb data
# server endpoint, username, password is found via a config file
f = h5pyd.File("/nrel/nsrdb/v3/nsrdb_2012.h5", 'r')

In [ ]:
list(f.attrs)  # list attributes belonging to the root group

In [ ]:
f.attrs['Version']   # attributes can be used to provide desriptions of the content

## Datasets

In [ ]:
list(f)  # list the datasets in the file

In [ ]:
# Datasets are stored in a 2d array of time x location
dset = f['ghi']
dset.shape

In [ ]:
# Extract datetime index for datasets
time_index = pd.to_datetime(f['time_index'][...].astype(str))
time_index # Temporal resolution is 30min

In [ ]:
f

In [ ]:
# Locational information is stored in either 'meta' or 'coordinates'
meta = pd.DataFrame(f['meta'][...])
meta.head()

In [ ]:
# Datasets have been saved as integers
dset.dtype

In [ ]:
dset.shape[0] * dset.shape[1] * 2 * 10**-9 # 70 GB per dataset!

In [ ]:
dset.chunks # Chunked by week

In [ ]:
dset.chunks[0] * dset.chunks[1] * 2 * 10**-6 # 2 MB per chunk

In [ ]:
# To convert dataset values back to floats use the 'psm_scale_factor'
dset.attrs['psm_scale_factor'] # Irradiance values have been truncated to integer precision

In [ ]:
# NOTE: this part of the example notebook wasn't working so I commented it out. --buck.

# # wind speed on the other hand has single decimal percision when scaled by 10
# scale_factor = f['wind_speed'].attrs['psm_scale_factor']
# units = f['wind_speed'].attrs['psm_units']
# print('wind_speed scale factor = ', scale_factor)
# print('wind_speed units after unscaling = ', units)
# f['wind_speed'][0, 0] / scale_factor # divide by scale_factor to return native value

## Time-slicing

Get the time_index from the server and convert to a pandas DatetimeIndex for convenience:

In [ ]:
time_index = pd.to_datetime(f['time_index'][...].astype(str))
time_index

Extract indexes for a particular span of time:

In [ ]:
march = time_index.month == 3
np.where(march)[0]

Or a particular date:

In [ ]:
timestep = np.where(time_index == '2012-07-04 00:00:00')[0][0]
timestep

## Map Data

In [ ]:
# Extract coordinates (lat, lon)
print(dict(f['coordinates'].attrs))
coords = f['coordinates'][...]

In [ ]:
dset = f['ghi']
data = dset[timestep, ::100]   # extract every 10th location at a particular time

In [ ]:
df = pd.DataFrame() # Combine data with coordinates in a DataFrame
df['longitude'] = coords[::100, 1]
df['latitude'] = coords[::100, 0]
df['ghi'] = data / dset.attrs['psm_scale_factor'] # unscale dataset

In [ ]:
df.shape

In [ ]:
df.plot.scatter(x='longitude', y='latitude', c='ghi',
                colormap='YlOrRd',
                title=str(time_index[timestep]))
plt.show()

In [ ]:
# Full resolution subset of Colorado
meta = pd.DataFrame(f['meta'][...])
CA = meta.loc[meta['state'] == b'California'] # Note .h5 saves strings as bit-strings
CA.head()

In [ ]:
data = dset[timestep][CA.index]  # full-resolution subset
df = CA[['longitude', 'latitude']].copy()
df['ghi'] = data / dset.attrs['psm_scale_factor']
df.shape

In [ ]:
df.plot.scatter(x='longitude', y='latitude', c='ghi',
                colormap='YlOrRd',
                title=str(time_index[timestep]))
plt.show()

## Nearest Timeseries for given Lat/Lon

In [ ]:
# Unlike the gridded WTK data the NSRDB is provided as sparse time-series dataset.
# The quickest way to find the nearest site it using a KDtree

dset_coords = f['coordinates'][...]
tree = cKDTree(dset_coords)
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

NewYorkCity = (40.7128, -74.0059)
NewYorkCity_idx = nearest_site(tree, NewYorkCity[0], NewYorkCity[1] )

print("Site index for New York City: \t\t {}".format(NewYorkCity_idx))
print("Coordinates of New York City: \t {}".format(NewYorkCity))
print("Coordinates of nearest point: \t {}".format(dset_coords[NewYorkCity_idx]))

In [ ]:
# Get the entire 2012 timeseries data for a point in NYC
%time tseries = dset[:, NewYorkCity_idx] / dset.attrs['psm_scale_factor']

In [ ]:
len(tseries)   # 1 years * 365 days * 24 hours * 30 minutes

In [ ]:
plt.plot(time_index, tseries)
plt.ylabel("ghi")
plt.title("NYC ghi in 2012")

## GHI Statistics

In [ ]:
df = pd.DataFrame({'ghi': tseries}, index=time_index)
df["year"] = df.index.year
df["month"] = df.index.month
df["day"] = df.index.day
df["hour"] = df.index.hour

agg = df.groupby(["month","hour"]).mean()
agg = agg.reset_index().pivot(index="month",columns="hour",values="ghi")
agg

In [ ]:
plt.imshow(agg)
plt.xlabel("Hour")
plt.ylabel("Month")
plt.title("12 x 24 Mean GHI (W/m^2)")
plt.colorbar()